In [2]:
import tensorflow as tf

In [3]:
a = tf.constant([1,2],name='a')
b = tf.constant([2,3],name='b')
result = a+b
sess = tf.Session()
sess.run(result)

array([3, 5])

计算图的使用


In [4]:
print(a.graph is tf.get_default_graph())

True


tf.Graph函数来生成新的计算图，不同计算图上的张量和运算都不会共享，以下代码示意了如何在不同的计算图上定义和使用变量

In [16]:
import tensorflow as tf
import numpy as np

g1 = tf.Graph()
with g1.as_default():
#     在计算图g1中定义变量'v',并设置初始值为0
    v = tf.get_variable('v',initializer=tf.zeros_initializer,shape=[1])
#     =tf.zeros_initializer()
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v',initializer=tf.ones_initializer(),shape=[1])
    
# 在计算图g1中读取变量'v1'的取值
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse = True):
        #在计算g1中，变量'v'的取值应该为0
        print(sess.run(tf.get_variable('v')))
        
# 在计算图g2中读取变量'v2'的取值
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("",reuse = True):
        print(sess.run(tf.get_variable('v')))


[0.]
[1.]


TensorFlow 使用GPU机制

In [19]:
g = tf.Graph()
# 指定计算运行的设备
with g.device('/gpu:0'):
    result = a+b

张量的概念


In [23]:
import tensorflow as tf
a= tf.constant([1,2],name='a')
b= tf.constant([2,3],name ='b')
result = tf.add(a,b,name ='add')
print(result)
#加上eval()就可以显示

Tensor("add_5:0", shape=(2,), dtype=int32)


In [24]:
import tensorflow as tf
a=tf.constant([1,2],name='a')
b=tf.constant([2.,3],name ='b')
c =a+b


ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("b_5:0", shape=(2,), dtype=float32)'

会话

In [25]:
sess =tf.Session()
sess.run(result)
sess.close()

In [27]:
#以上相当于
with tf.Session() as sess:
    sess.run(pass)

SyntaxError: invalid syntax (<ipython-input-27-700ea603ac83>, line 3)

In [30]:
sess = tf.Session()
with sess.as_default():
    print(result.eval())
    
# 相同功能的代码
sess = tf.Session()
#法二
print(sess.run(result))
#法三
print(result.eval(session=sess))

[3 5]
[3 5]
[3 5]


In [31]:
# InteractiveSession可以省去将产生的对话注册为默认对话的过程
sess = tf.InteractiveSession()
print(result.eval())
sess.close()

[3 5]


In [33]:
# ConfigProto配置会话,类似并行的线程数，GPU分配策略，运算超时时间等
# allow_soft_placement为True,GPU不行时就取CPU执行
# log_device_placement 为True时将会记录每个节点被安排在哪个设备上以方便调试，一般设为False以减少日志量
config = tf.ConfigProto(allow_soft_placement = True,
                       log_device_placement = True)
sess1 = tf.InteractiveSession(config=config)
sess2 = tf.Session(config=config)

Tensorflow 实现神经网络

In [36]:
#前向传播
import tensorflow as tf
w1 = tf.Variable(tf.random_normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal((3,1),stddev=1,seed=1))

x = tf.constant([[0.7,0.9]])
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()
sess.run(w1.initializer)
sess.run(w2.initializer)

print(sess.run(y))
sess.close()

[[3.957578]]


In [40]:
#张量具有名字，维度和类型三个属性
w1 = tf.Variable(tf.random_normal([2,3],stddev =1),name='w1')
w2 = tf.Variable(tf.random_normal([2,2],stddev=1),name='w2')
# tf.assign(w1,w2) 会报维度不匹配错误
#解决维度不匹配错误
tf.assign(w1,w2,validate_shape=False)

<tf.Tensor 'Assign_2:0' shape=(2, 2) dtype=float32_ref>

In [43]:
#tensorflow实现反向传播
import tensorflow as tf
w1 = tf.Variable(tf.random_normal([2,3],stddev=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1))

#定义Placehold作为存放数据的地方，如果维度时确定的，那么给出的维度可以降低出错的概率
x = tf.placeholder(tf.float32,shape=(1,2),name='input')
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess =tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# sess.run(y) 报错，没有输入初始值
print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))


[[0.6253257]]


In [ ]:
#损失函数+反向传播
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0))
                               +(1-y)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
#定义学习率
learning_rate = 0.001
#定义反向传播算法优化参数
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [53]:
#完整神经网络样例
import tensorflow as tf
from numpy.random import RandomState

# 定义训练数据的batch大小
batch_size = 8

#定义神经网络的参数
w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_ = tf.placeholder(tf.float32,shape=(None,1),name='y-input')

#定义神经网络的前向传播
a = tf.matmul(x,w1)
y=tf.matmul(a,w2)

#定义损失函数和反向传播算法
y= tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1))
                               +(1-y)*tf.log(tf.clip_by_value(1-y,1e-10,1)))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)

#通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size =128
X = rdm.rand(dataset_size,2)
Y = [[int(x1 + x2 < 1)] for (x1,x2) in X]

# 创建会话
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w1))
    print(sess.run(w2))
    
    #设定训练的次数
    STEPS = 50
    for i in range(STEPS):
#         每次选取batch_size各样本进行训练
        start = (i*batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        
#         通过选取的样本训练神经网络并更新参数
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        if i % 100 ==0:
#             每隔一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print('After %d training step(s) ,cross_entropy on all data is %g'%(i,total_cross_entropy))
        
    print(sess.run(w1))
    print(sess.run(w2))
#     print(sess.run(y_,feed_dict={x:[[1.,2.],[1.,2.],[1.,2.],[1.,2.],[1.,2.],[1.,2.],[1.,2.],[1.,2.]]}))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s) ,cross_entropy on all data is 0.308078
[[-1.2382916   1.9015338   0.57100815]
 [-2.876326    0.52161485  1.1146498 ]]
[[-1.2342769]
 [ 1.9215373]
 [ 0.5291113]]


InvalidArgumentError: You must feed a value for placeholder tensor 'y-input_6' with dtype float and shape [?,1]
	 [[Node: y-input_6 = Placeholder[dtype=DT_FLOAT, shape=[?,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'y-input_6', defined at:
  File "C:\Program Files (x86)\software\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files (x86)\software\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Program Files (x86)\software\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Program Files (x86)\software\Anaconda\lib\site-package